In [7]:
import os
import tarfile
import urllib
import shutil
import json
import random
import numpy as np
import boto3
import sagemaker

from tqdm import tqdm
from sagemaker.amazon.amazon_estimator import get_image_uri
from matplotlib import pyplot as plt

role = sagemaker.get_execution_role()
print(role)

sess = sagemaker.Session()
print(sess)

arn:aws:iam::108782086480:role/service-role/AmazonSageMaker-ExecutionRole-20241028T231313


# Sagemaker data preparation

In [ ]:
sess = sagemaker.Session()

s3_train_path = sess.upload_data(path='train', bucket=bucket_name, key_prefix='train')
s3_train_lst_path = sess.upload_data(path='train_lst', bucket=bucket_name, key_prefix='train_lst')
s3_validation_path = sess.upload_data(path='validation', bucket=bucket_name, key_prefix='validation')
s3_validation_lst_path = sess.upload_data(path='validation_lst', bucket=bucket_name, key_prefix='validation_lst')
print(s3_train_path, s3_train_lst_path, s3_validation_path, s3_validation_lst_path)


# Sagemaker Estimator

In [49]:
model = sagemaker.estimator.Estimator(
    image_uri="108782086480.dkr.ecr.us-east-1.amazonaws.com/simple-docker-service-0e4ed6c4db93:latest",
    role=role,
    instance_count=1,
    instance_type='ml.m5.large',
    volume_size=5,
    input_mode='File',
    output_path='s3://mnist-20241029/output',
    sagemaker_session=sess
)

In [50]:
model.set_hyperparameters(
    num_layers=18,
    use_pretrained_model=1,
    image_shape='3,224,224',
    num_classes=37,
    mini_batch_size=32,
    resize=224,
    epochs=10,
    learning_rate=0.001,
    num_training_samples=10000,
    use_weighted_loss=1,
    augmentation_type='crop_color_transform',
)

In [51]:
data_channels = {
    'train': 's3://mnist-20241029/t10k-images-idx3-ubyte',
    'validation': 's3://mnist-20241029/t10k-images-idx3-ubyte'
}

In [52]:
model.fit(inputs=data_channels, logs=True)

INFO:sagemaker:Creating training-job with name: simple-docker-service-0e4ed6c4db93-2024-10-29-13-41-51-270


2024-10-29 13:41:52 Starting - Starting the training job...
2024-10-29 13:42:07 Starting - Preparing the instances for training...
2024-10-29 13:42:32 Downloading - Downloading input data...
2024-10-29 13:42:57 Downloading - Downloading the training image.........
2024-10-29 13:44:49 Training - Training image download completed. Training in progress..A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.
If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.
Traceback (most recent call last):  File "/app/mnist_cnn.py", line 1, in <module>
    import torch
  File "/usr/local/lib/python3.9/site-packages/torch/__init__.py", line 1477, in <module>
    from .functional im

UnexpectedStatusException: Error for Training job simple-docker-service-0e4ed6c4db93-2024-10-29-13-41-51-270: Failed. Reason: AlgorithmError: , exit code: 1

# model deploy

In [ ]:
predictor = model.deploy(
    instance_type='ml.m5.large',
    initial_instance_count=1,
    image_uri='your-custom-image-uri'
)

In [ ]:
# sagemaker.Session().delete_endpoint(predictor.endpoint)